## **Dependencies**

In [8]:
import pandas as pd
from tqdm.notebook import tqdm

## **Load JSON**

Choose respective json file to transform to data frame

Json be found in /src/response_[ORG]

### Define Params

In [37]:
src_folder = "../../src/"
response_file = src_folder + "response_bmz.json" # json format -> fetched in iati.ipynb
output_file = src_folder + "transformed_bmz.csv"

In [38]:
#Read in data into df from json
df = pd.read_json(response_file)
df.head(10)

,sector_code,iati_identifier,title_narrative,document_link_url,reporting_org_ref,sector_vocabulary,activity_date_type,activity_status_code,description_narrative,last_updated_datetime,activity_date_iso_date,recipient_country_code,location_name_narrative,reporting_org_narrative,title_narrative_xml_lang,description_narrative_xml_lang,recipient_region_code
0,[12230],DE-1-201629492-0,[Basisgesundheitsdienste zur Senkung der Mütte...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[Basisgesundheitsdienste zur Senkung der Mütte...,2024-01-22T00:00:00Z,"[2016-09-01T00:00:00Z, 2016-09-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",NaN
1,[43040],DE-1-201673854-0,[Nachhaltige Wasserversorgung und Lebensunterh...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[Nachhaltige Wasserversorgung und Lebensunterh...,2024-01-22T00:00:00Z,"[2016-04-01T00:00:00Z, 2016-04-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",NaN
2,[11330],DE-1-201676782-0,[Wirtschaftliche Selbständigkeit junger Frauen...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[Wirtschaftliche Selbständigkeit junger Frauen...,2024-01-22T00:00:00Z,"[2017-01-01T00:00:00Z, 2017-01-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",NaN
3,[15220],DE-1-201676790-0,[Stärkung des Gemeindelebens und Friedensbildu...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[Stärkung des Gemeindelebens und Friedensbildu...,2024-01-22T00:00:00Z,"[2017-01-01T00:00:00Z, 2017-01-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",NaN
4,[11130],DE-1-201676808-0,"[Primarschullehrerausbildung Südsudan, Fortfüh...","[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,"[Primarschullehrerausbildung Südsudan, Fortfüh...",2024-01-22T00:00:00Z,"[2017-01-01T00:00:00Z, 2017-01-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",NaN
5,[31130],DE-1-201701127-2712,[SEWOH - Projekt zur Ernährungssicherung von b...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[SEWOH - Projekt zur Ernährungssicherung von b...,2024-01-22T00:00:00Z,"[2018-01-01T00:00:00Z, 2018-01-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",NaN
6,[14030],DE-1-201706019-2492,[Verbesserte Grundversorgung hinsichtlich WASH...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",4,[Verbesserte Grundversorgung hinsichtlich WASH...,2024-01-22T00:00:00Z,"[2017-09-01T00:00:00Z, 2017-09-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",NaN
7,[12230],DE-1-201706019-2695,[Aufbau der Basisgesundheitsversorgung in Make...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",4,[Aufbau der Basisgesundheitsversorgung in Make...,2024-01-22T00:00:00Z,"[2017-10-01T00:00:00Z, 2017-10-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",NaN
8,[12220],DE-1-201706019-2836,[Berufsausbildung und Einkommensmöglichkeiten ...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",4,[Berufsausbildung und Einkommensmöglichkeiten ...,2024-01-22T00:00:00Z,"[2017-11-01T00:00:00Z, 2017-11-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",NaN
9,"[12240, 14020, 12220]",DE-1-201718568,[&quot;Stärkung sozialer Grunddienste für vuln...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,"[1, 1, 1]","[1, 2, 3, 4]",4,"[Ziel des Einzelvorhabens ist es, durch eine V...",2023-12-31T00:00:00Z,"[2017-11-23T

In [10]:
print(f"Entries: {len(df)}")
df.reporting_org_ref.value_counts()

Entries: 31613


DE-1    31613
Name: reporting_org_ref, dtype: int64

In [11]:
# create new empty df to fill with transformed data

trans_df = pd.DataFrame()

## **Feature Transformation & Engineering****

### IATI ID

In [12]:
trans_df["iati_id"] = df["iati_identifier"].values

trans_df.head(2)

,iati_id
0,DE-1-201629492-0
1,DE-1-201673854-0


### Title

In [13]:
################
# Add en title #
################

pbar = tqdm(total=len(df))
trans_df["title_en"] = "NaN"

for index, row in df.iterrows():
    lang_list = row['title_narrative_xml_lang']
    title_row = row['title_narrative']

    # nan in pandas is type float
    # check if nan and if yes take first entry in lang
    try:
        if isinstance(lang_list, float):
            if isinstance(title_row, float):
                trans_df["title_en"][index] = "NaN"
            else:
                trans_df["title_en"][index] = title_row[0]
        elif len(lang_list) == len(title_row):
            for j in range(0, len(lang_list)):
                if "en" or "EN" in lang_list:
                    if lang_list[j].lower() == "en":
                        title = title_row[j]
                        trans_df["title_en"][index] = title
                else:
                    trans_df["title_en"][index] = "NaN"
        else:
            pass
    except:
        print(f"Error: Index: {index}, Row: {lang_list}, {title_row}")
    
    # Update the progress bar
    pbar.update(1)

# Close the progress bar
pbar.close()

trans_df.head(2)

  0%|          | 0/31613 [00:00<?, ?it/s]

,iati_id,title_en
0,DE-1-201629492-0,Basic health services with focus on reducing m...
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...


In [14]:
###################
# Add other title #
###################

pbar = tqdm(total=len(df))

trans_df["title_other"] = "NaN"

for index, row in df.iterrows():
    try:
        lang_list = row['title_narrative_xml_lang']

        # every title which is has no lang attribute is classified as English and therefore not in other
        if isinstance(lang_list, float):
            trans_df["title_other"][index] = "NaN"
        elif len(lang_list) == len(title_row):
            for j in range(0, len(lang_list)):
                if lang_list[j].lower() != "en":
                    title = row['title_narrative'][j]
                    if trans_df["title_other"][index] == "NaN":
                        trans_df["title_other"][index] = title
                    else:
                        trans_df["title_other"][index] = f"{trans_df['title_other'][index]}; {title}"
        else:
            trans_df["title_other"][index] = title
        
        # Update the progress bar
        pbar.update(1)
        
    except Exception as e:
        print(f"Error: Index: {index} \n Row: {row} \n Exception: {e}")

# Close the progress bar
pbar.close()

trans_df.head(200)

  0%|          | 0/31613 [00:00<?, ?it/s]

Error: Index: 10629 
 Row: sector_code                                                                 [33130]
iati_identifier                                                    DE-1-202120004-0
title_narrative                   [Stärkung der Qualitätsinfrastruktur zur Unter...
document_link_url                                                               NaN
reporting_org_ref                                                              DE-1
sector_vocabulary                                                               [1]
activity_date_type                                                        [1, 2, 3]
activity_status_code                                                              2
description_narrative             [Stärkung der Qualitätsinfrastruktur zur Unter...
last_updated_datetime                                          2023-12-31T00:00:00Z
activity_date_iso_date            [2023-03-01T00:00:00Z, 2023-03-01T00:00:00Z, 2...
recipient_country_code                           

,iati_id,title_en,title_other
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt
2,DE-1-201676782-0,Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...
3,DE-1-201676790-0,Community Empowerment and Peace Building,Stärkung des Gemeindelebens und Friedensbildun...
4,DE-1-201676808-0,Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführung"
...,...,...,...
195,DE-1-201334390-0,Vocational Training in Sumbe and Porto Amboim,"Berufsausbildung in Sumbe und Porto Amboim, Pr..."
196,DE-1-201339027-0,Appropriate and ecological building technologi...,Beratung von Partnerorganisationen in angepass...
197,DE-1-201339258-0,Integrated rural and sustainable development f...,Integrierte ländliche Entwicklung durch Selbst...
198,DE-1-201339324-0,Integrated rural and sustainable development f...,Integrierte Förderung nachhaltiger kleinbäuerl...


### Organization

In [15]:
trans_df['organization'] = df['reporting_org_narrative'].apply(lambda x: x[0])

print(trans_df.organization.value_counts())
trans_df.head(2)

Bundesministerium für wirtschaftliche Zusammenarbeit und Entwicklung (BMZ)    31538
Federal Ministry for Economic Cooperation and Development (BMZ)                  75
Name: organization, dtype: int64


,iati_id,title_en,title_other,organization
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...


### Country

In [16]:
trans_df["country"] = df["recipient_country_code"]
trans_df.head(5)

,iati_id,title_en,title_other,organization,country
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...,[SS]
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...,[SS]
2,DE-1-201676782-0,Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...,Bundesministerium für wirtschaftliche Zusammen...,[SS]
3,DE-1-201676790-0,Community Empowerment and Peace Building,Stärkung des Gemeindelebens und Friedensbildun...,Bundesministerium für wirtschaftliche Zusammen...,[SS]
4,DE-1-201676808-0,Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführung",Bundesministerium für wirtschaftliche Zusammen...,[SS]


### Region

In [17]:
trans_df['region'] = df['recipient_region_code']
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN
2,DE-1-201676782-0,Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN
3,DE-1-201676790-0,Community Empowerment and Peace Building,Stärkung des Gemeindelebens und Friedensbildun...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN
4,DE-1-201676808-0,Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführung",Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN


### Location

In [18]:
trans_df['location'] = df['location_name_narrative']
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region,location
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba]
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba]
2,DE-1-201676782-0,Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba]
3,DE-1-201676790-0,Community Empowerment and Peace Building,Stärkung des Gemeindelebens und Friedensbildun...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba]
4,DE-1-201676808-0,Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführung",Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba]


### Descriptions

In [19]:
#################################
# Description english and other #
#################################

trans_df["description_en"] = "NaN"
trans_df["description_other"] = "NaN"

for index, row in df.iterrows():

    try:
        descr_list = row['description_narrative_xml_lang']
        descr_row = row['description_narrative']

        # nan in pandas is type float
        # check if nan and if yes take first entry in descr
        if isinstance(descr_list, float):
            if isinstance(descr_row, float):
                trans_df["description_en"][index] = "NaN"
            else:
                trans_df["description_en"][index] = descr_row[0]
        else:
            for j in range(0, len(descr_list)):
                if descr_list[j].lower() == "en":
                    if type(descr_row) == float:
                        descr = "NaN"
                    else:
                        descr = descr_row[j]
                    if trans_df["description_en"][index] == "NaN":
                        trans_df["description_en"][index] = descr
                    else:
                        trans_df["description_en"][index] = f"{trans_df['description_en'][index]}; {descr}"
                else:
                    if type(descr_row) == float:
                        descr = "NaN"
                    else:
                        descr = descr_row[j]
                    if trans_df["description_other"][index] == "NaN":
                        trans_df["description_other"][index] = descr
                    else:
                        trans_df["description_other"][index] = f"{trans_df['description_other'][index]}; {descr}"
    except:
        print(f"Error: Index: {index} \n Row: {row}")
        
trans_df.head(5)

Error: Index: 2023 
 Row: sector_code                                                                 [43060]
iati_identifier                                                    DE-1-201298330-0
title_narrative                   [Anpassung an den Klimawandel in Nord Burundi,...
document_link_url                                                               NaN
reporting_org_ref                                                              DE-1
sector_vocabulary                                                               [1]
activity_date_type                                                     [1, 2, 3, 4]
activity_status_code                                                              3
description_narrative             [Anpassung an den Klimawandel in Nord Burundi,...
last_updated_datetime                                          2024-01-22T00:00:00Z
activity_date_iso_date            [2013-01-01T00:00:00Z, 2013-01-01T00:00:00Z, 2...
recipient_country_code                            

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterha...
2,DE-1-201676782-0,Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...
3,DE-1-201676790-0,Community Empowerment and Peace Building,Stärkung des Gemeindelebens und Friedensbildun...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Community Empowerment and Peace Building; This...,Stärkung des Gemeindelebens und Friedensbildun...
4,DE-1-201676808-0,Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführung",Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführ..."


### Status

In [21]:
# https://iatistandard.org/en/iati-standard/203/codelists/activitystatus/
activity_status = {
    1: "Pipeline/identification",
    2: "Implementation",
    3: "Finalisation",
    4: "Closed",
    5: "Cancelled",
    6: "Suspended"
}

trans_df["status"] = df.activity_status_code
trans_df['status'] = trans_df['status'].replace(activity_status)

trans_df.head(2)
    

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Finalisation
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterha...,Finalisation


### Date

In [22]:
#############
# Date Type #
#############

# One Hot
# 1 -> Yes
# 0 -> No

# Codes:
# 1 Planned start
# 2 Actual start
# 3 Planned end
# 4 Actual end

trans_df["planned_start"] = "NaN"
trans_df["actual_start"] = "NaN"
trans_df["planned_end"] = "NaN"
trans_df["actual_end"] = "NaN"

date_types = {
    1: "planned_start",
    2: "actual_start",
    3: "planned_end",
    4: "actual_end"
}

for index, row in df.iterrows():
    dtype_list = row["activity_date_type"]
    iso_date_list = row["activity_date_iso_date"]

    combined_list = list(zip(dtype_list, iso_date_list))

    # replace nums with column names from date_types
    combined_list = [(date_types[int(t[0])], t[1]) for t in combined_list]

    for i in combined_list:
        trans_df[i[0]] = i[1]

trans_df.head(1800)


,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterha...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z
2,DE-1-201676782-0,Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z
3,DE-1-201676790-0,Community Empowerment and Peace Building,Stärkung des Gemeindelebens und Friedensbildun...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Community Empowerment and Peace Building; This...,Stärkung des Gemeindelebens und Friedensbildun...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z
4,DE-1-201676808-0,Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführung",Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführ...",Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,DE-1-201874593-0,Sexual violence prevention and protection for ...,Schutz vor sexueller Gewalt besonders gefährde...,Bundesministerium für wirtschaftliche Zusammen...,[VN],NaN,[Hanoi],Sexual violence prevention and protection for ...,Schutz vor sexueller Gewalt besonders gefährde...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z
1796,DE-1-201875343-0,Community Management in New Rural Development ...,Beteiligung von Gemeinden im Rahmen des Neuen ...,Bundesministerium für wirtschaftliche Zusammen...,[VN],NaN,[Hanoi],Community Management in New Rural Development ...,Beteiligung von Gemeinden im Rahmen des Neuen ...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z
1797,DE-1-201876044-0,capacity building on financial management,Kapazitätsförderung des Partnerfeldes mit Schw...,Bundesministerium für wirtschaftliche Zusammen...,[VN],NaN,[Hanoi],capacity building on financial management; cap...,Kapazitätsförderung des Partnerfeldes mit Schw...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z
1798,DE-1-201906015-4134,Associated agriculture for long-term income st...,Assoziierte Viehwirtschaft für eine langfristi...,Bundesministerium für wirtschaftliche Zusammen...,[VN],NaN,[Hanoi],Associated agriculture for long-term income st...,Assoziierte Viehwirtschaft für eine langfristi...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z


### Last Update

In [23]:
trans_df['last_update'] = df['last_updated_datetime']
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterha...,Finalisation,2016-06-14T00:00:00Z,2016-06-14T00:00:00Z,2016-12-31T00:00:00Z,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z


### Sector Codes (CRS & SGDs)

In [24]:
sector_codes = {
    1: "OECD DAC CRS Purpose Codes (5 digit)",
    2: "OECD DAC CRS Purpose Codes (3 digit)",
    3: "Classification of the Functions of Government (UN)",
    4: "Statistical classification of economic activities in the European Community",
    5: "National Taxonomy for Exempt Entities (USA)",
    6: "AidData",
    7: "SDG Goal",
    8: "SDG Target",
    9: "SDG Indicator",
    10: "Humanitarian Global Clusters (Inter-Agency Standing Committee)",
    11: "North American Industry Classification System (NAICS)",
    12: "UN System Function",
    99: "Reporting Organisation", # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity
    98: "Reporting Organisation 2" # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity (if they are referencing more than one)
}

In [25]:
crs5_df = pd.read_csv("../../src/crs5_codes.csv")
crs5_df.head(2)

,code,name,description,language,category,category-name,category-description
0,11110,Education policy and administrative management,"Education sector policy, planning and programm...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...
1,11120,Education facilities and training,"Educational buildings, equipment, materials; s...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...


In [26]:
crs3_df = pd.read_csv("../../src/crs3_codes.csv")
crs3_df.head(2)

,code,name,description,language,category,category-name,category-description
0,111,"Education, Level Unspecified",The codes in this category are to be used only...,en,NaN,NaN,NaN
1,112,Basic Education,NaN,en,NaN,NaN,NaN


In [27]:
sgd_goals_df = pd.read_csv("../../src/sgd_goals.csv")
sgd_goals_df.head(2)

,code,name,description,language,category,category-name,category-description
0,1,Goal 1. End poverty in all its forms everywhere,NaN,en,NaN,NaN,NaN
1,2,"Goal 2. End hunger, achieve food security and ...",NaN,en,NaN,NaN,NaN


In [28]:
sgd_targ_df = pd.read_csv("../../src/sgd_targets.csv")
sgd_targ_df.head(2)

,code,name,description,language,category,category-name,category-description
0,1.1,"By 2030, eradicate extreme poverty for all peo...",NaN,en,NaN,NaN,NaN
1,1.2,"By 2030, reduce at least by half the proportio...",NaN,en,NaN,NaN,NaN


In [29]:
# to process different variants of codes (crs3, crs5 etc.)

def process_codes(combined_list, translation_df, code_index):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                translation = translation_df.loc[translation_df['code'] == int(i[1]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"

In [30]:
###############
# ECTRACT CRS #
###############
trans_df["crs_5_code"] = "NaN"
trans_df["crs_5_name"] = "NaN"

trans_df["crs_3_code"] = "NaN"
trans_df["crs_3_name"] = "NaN"

trans_df["sgd_goal_code"] = "NaN"
trans_df["sgd_goal_name"] = "NaN"

trans_df["sgd_target_code"] = "NaN"
trans_df["sgd_target_name"] = "NaN"

#trans_df["sgd_indicator_code"] = "NaN"
#trans_df["sgd_indicator_name"] = "NaN"

for index, row in df.iterrows():
    crs_voc_list = row['sector_vocabulary']
    crs_code_list = row['sector_code']

    if type(crs_voc_list) == float:
        pass
    else:
        try:
            combined_list = list(zip(crs_voc_list, crs_code_list))

            # CRS 5
            crs5_str, crs5_codes = process_codes(combined_list, crs5_df, "1")
            trans_df["crs_5_code"][index] = crs5_codes
            trans_df["crs_5_name"][index] = crs5_str

            # CRS 3
            crs3_str, crs3_codes = process_codes(combined_list, crs3_df, "2")
            trans_df["crs_3_code"][index] = crs3_codes
            trans_df["crs_3_name"][index] = crs3_str

            # SGD Goals
            sgd_goal_str, sgd_goal_codes = process_codes(combined_list, sgd_goals_df, "7")
            trans_df["sgd_goal_code"][index] = sgd_goal_codes
            trans_df["sgd_goal_name"][index] = sgd_goal_str

            # SGD Target
            sgd_targ_str, sgd_targ_codes = process_codes(combined_list, sgd_targ_df, "8")
            trans_df["sgd_target_code"][index] = sgd_targ_codes
            trans_df["sgd_target_name"][index] = sgd_targ_str

            ## SGD Indicator
            #sgd_ind_str, sgd_ind_codes = process_codes(combined_list, sgd_ind_df, "9")
            #trans_df["sgd_indicator_code"][index] = sgd_ind_codes
            #trans_df["sgd_indicator_name"][index] = sgd_ind_str
        except:
            print(f"Error on Index {index}")
            pass

trans_df.head(100)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,...,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,sgd_goal_code,sgd_goal_name,sgd_target_code,sgd_target_name
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Finalisation,...,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z,12230;,Basic health infrastructure;,NaN,NaN,NaN,NaN,NaN,NaN
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterha...,Finalisation,...,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z,43040;,Rural development;,NaN,NaN,NaN,NaN,NaN,NaN
2,DE-1-201676782-0,Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Practical Vocational Skills Training with Pers...,Wirtschaftliche Selbständigkeit junger Frauen ...,Finalisation,...,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z,11330;,Vocational training;,NaN,NaN,NaN,NaN,NaN,NaN
3,DE-1-201676790-0,Community Empowerment and Peace Building,Stärkung des Gemeindelebens und Friedensbildun...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Community Empowerment and Peace Building; This...,Stärkung des Gemeindelebens und Friedensbildun...,Finalisation,...,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z,15220;,"Civilian peace-building, conflict prevention a...",NaN,NaN,NaN,NaN,NaN,NaN
4,DE-1-201676808-0,Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführung",Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Training Quality Teachers for the development ...,"Primarschullehrerausbildung Südsudan, Fortführ...",Finalisation,...,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z,11130;,Teacher training;,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DE-1-202218295-0,This information is not available for this pro...,Diese Information ist für diese Maßnahme nicht...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],This information is not available for this pro...,Diese Information ist für diese Maßnahme nicht...,Implementation,...,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z,31120; 14030; 15220; 43060;,Agricultural development; Basic drinking water...,NaN,NaN,NaN,NaN,NaN,NaN
96,DE-1-202218394-0,This information is not available for this pro...,Diese Information ist für diese Maßnahme nicht...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],This information is not available for this pro...,Diese Information ist für diese Maßnahme nicht...,Implementation,...,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z,43072; 15180; 14031; 11110;,Household food security programmes; Ending vio...,NaN,NaN,NaN,NaN,NaN,NaN
97,DE-1-202218626-0,Interm Country Strategic Plan (ICSP),Interm Country Strategic Plan (ICSP),Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Interm Country Strategic Plan (ICSP); • SO2: V...,Interm Country Strategic Plan (ICSP); • SO2: G...,Finalisation,...,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z,16050;,Multisector aid for basic social services;,NaN,NaN,NaN,NaN,NaN,NaN
98,DE-1-202226082-0,This information is not available for this pro...,Diese Information ist für diese Maßnahme nicht...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],This information is not available for this pro...,Diese Information ist für diese Maßnahme nicht...,Implementation,...,2016-12-31T00:00:00Z,2024-01-22T00:00:00Z,43060;,Disaster Risk Reducti

### Documents

In [31]:
df.document_link_url.value_counts()

trans_df['docs'] = df['document_link_url']
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,...,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,sgd_goal_code,sgd_goal_name,sgd_target_code,sgd_target_name,docs
0,DE-1-201629492-0,Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Basic health services with focus on reducing m...,Basisgesundheitsdienste zur Senkung der Mütter...,Finalisation,...,2024-01-22T00:00:00Z,12230;,Basic health infrastructure;,NaN,NaN,NaN,NaN,NaN,NaN,"[https://www.bmz.de/de/laender/suedsudan, http..."
1,DE-1-201673854-0,Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterhalt,Bundesministerium für wirtschaftliche Zusammen...,[SS],NaN,[Juba],Sustainable Water Sources and Livelihoods amon...,Nachhaltige Wasserversorgung und Lebensunterha...,Finalisation,...,2024-01-22T00:00:00Z,43040;,Rural development;,NaN,NaN,NaN,NaN,NaN,NaN,"[https://www.bmz.de/de/laender/suedsudan, http..."


## **Save DF as CSV**

In [39]:
df.to_csv(output_file, index=False) 